version 0.0.2.9.1

TODO:
  * add collection of .ig files, WW, GammaGamma events included
  * Add collection of .ig files that do not contain events

In [ ]:
import sys
sys.path.append("..")

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

# Events
------------------

In the following, partly covered by a colloquium, the origin of the data sets used in the following will be examined in more detail: the CMS detector. The construction of the detector and the functionality of individual detector components will be discussed. In addition, events are to be analyzed and important physical quantities should be displayed visually. For both cases the web interface [Ispy-WebGL](https://github.com/cms-outreach/ispy-webgl) can be used.

In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
Identify and name all decays from the local collection and save the representations of some of them.
</div>

With an Internet connection:

In [ ]:
%%html
<iframe src="https://ispy-webgl.web.cern.ch/ispy-webgl/" width="100%" height="700"></iframe>

Without an internet connection: Open the `index.html` from the [Github Repository](https://github.com/cms-outreach/ispy-webgl) locally in a web browser.

In the following, only events from the decay into four leptons (muons and electrons, not tau leptons) are considered.

# Data format
------------------------

The data format used in the following, from which all necessary sizes are taken, is a modified .CSV format. The advantage of this format is that the data can be viewed at any time in the same way as a human would read it.

This section serves to familiarize with the data format in order to be able to perform the task in the following section.

The separation of individual variables in an event is done using ";". The entries of the individual leptons within an event, on the other hand, are classically separated with ",". This gives the advantage of individual numbers of leptons in an event without the introduction of additional placeholders.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

name_1 = "data/mc_init/MC_2012_ZZ_to_4L_2el2mu_init.csv"
name_2 = "data/mc_init/MC_2012_ZZ_to_4L_4mu_init.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

For processing individual sizes, the respective elements from the string (`str`) data format can be converted back into a list (`list`):
For this purpose the options are available either the standard library `ast`

In [ ]:
import ast
px = ast.literal_eval(f"[{dataframe_2.loc[3, 'px']}]")
px = np.array(px, dtype=float)
px

or to use the `split` method for the `str`

In [ ]:
py = dataframe_2.loc[3, 'py'].split(",")
py = np.array(py, dtype=float)
py

From this, new quantities such as the transverse momentum can be determined, on the basis of which a limitation in the quantities can and should be made

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

For muons, a minimum value of 5 GeV for the transverse impulse must be exceeded. For transverse impulse values below this, the probability of muon misidentification rises.
In this respect, all muons in this event that do not meet the condition must be rejected.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

# Application of the filters by using custom Apply Class
-----------------------------------

A filter is applied to all variables within an event and is different from the filter for the next event.
The application of the filter to the data set in series is already implemented and can be applied via the Apply class.

In [ ]:
from include.processing.Apply import Apply

When the Apply object is created, an "allowed" and a "calculation" class instance must also be passed. The required variables or filters for the application of the filters are calculated using these classes.
For the example of the minimum transverse impulse the two classes can be represented as a collection of functions, which could stand alone (`@staticmethod`), but logically combined in one (two) class(es).

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Allowed_Start(object):
    
    @staticmethod
    def min_pt(pt, look_for):
        return pt > 7.0 if look_for == "electron" else (
            pt > 5.0 if look_for == "muon" else None)

In [ ]:
print(name_2)
process = Apply(input_=name_2, particle_type="muon", use_n_rows=10000,
                calc_instance=Calc_Start, allowed_instance=Allowed_Start)

In the data set `name_2` the variable of the transverse pulse does not yet exist. In an intermediate step, which is already contained in the filter `"check_min_pt"`, the transverse pulse can also be added explicitly:

In [ ]:
process.add_variable("pt")
process.data

The histogram of such a variable can be displayed by the method `hist_of_variable`.

In [ ]:
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Applying the filter for the minimum transverse impulse and removing all subsequent events containing less than four leptons provides the appropriate distribution:

In [ ]:
process.filter(filter_name="check_min_pt")
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Other variables can also be visualized by the histogram.
The performed cut mainly influences the distribution of the transverse impulse, but changes little in the distribution of other variables.

<div class="alert alert-info">
Examine the distributions of some of the quantities present in the records.

</div>

In [1]:
# here goes the code

# Calculation of important variables
--------------------------------

<div class="alert alert-info">

Following the example of the transversal impulse, implement all subsequently necessary quantities and visualize them appropriately:
  * Pseudorapidity $\eta$
  * Transverse pulse $p_T$
  * Azimuthal angle $\phi$

And explain your observations.

(Check their implementations using MC simulations of the background)
</div>

The class skeleton used for this inherits the previous method for the transverse momentum and an initial class containing methods such as the reconstruction of Z-boson pairs are added afterwards.

In [ ]:
class CalcStudent(Calc_Start):
    '''
    Class for the calculation of certain sizes that are used for
    the cuts or are essential for the reconstruction.
    '''

    @staticmethod
    def combined_charge(charge, combine_num):
        '''
        Tests whether an electrically neutral charge combination is possible.

        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combine_num: int
                            4 if look_for is not "both", 2 else
        :return: bool
        '''
        # code

    @staticmethod
    def eta(px=None, py=None, pz=None, energy=None):
        '''
        Calculates the pseudorapidity.
        Optional with or without energy.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None, eta=None, phi=None):
        '''
        Calculates the square of the invariant mass.
        Optional with or without energy.
        Optionally with or without eta and phi.

        :param phi: ndarray
                    1D array containing data with "float" type.
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def phi(px, py):
        '''
        Calculation of the angle phi.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_phi(phi1, phi2):
        '''
        Calculation of the difference between two phi angles.

        :param phi1: ndarray
                     1D array containing data with "float" type.
        :param phi2: ndarray
                     1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_r(eta, phi):
        '''
        Calculation of delta_r.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param phi: ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

# Creating the cuts
------------------------------

Analogous to the cut to the minimum transverse pulse, further cuts can be introduced which are then used as well.

<div class="alert alert-info">

Implement the methods listed in the class `AllowedStudent`. Estimate the choice of cuts using the distributions in the chapter **Application of the filters by using custom Apply Class**, the **detector** and the **kinematic constraints** of the events.

(Check your implementations using the MC simulations of the background).
    
</div>

In [ ]:
class AllowedStudent(Allowed_Start):
    '''
    Class that introduces certain cuts and thus restricts the leptons in the events.
    '''
    
    @staticmethod
    def delta_r(delta_r):
        '''
        Checks if delta_r is smaller than the allowed value.

        :param delta_r: ndarray
                        1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code

    @staticmethod
    def rel_pf_iso(rel_pf_iso):
        '''
        Checks if rel_pf_iso is smaller than the allowed value.

        :param rel_pf_iso: ndarray
                           1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def misshits(misshits):
        '''
        Checks if the minimum number of misshits was kept.

        :param misshits:
        :return:
        '''
        # code
        
    @staticmethod
    def pt(p_t, look_for, coll_size=4):
        '''
        Checks if the exact pedingun regarding pt is observed.
        (>20 GeV: >= 1; >10 GeV: >= 2; >Minimum pt: >= 4).

        :param p_t: ndarray
                    1D array containing data with `float` type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :param coll_size: int
                          4 if look_for is not "both", 2 else
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def eta(eta, look_for):
        '''
        Checks if the pseudorapidity of leptons is valid.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def lepton_type(typ, look_for):
        '''
        Checks for the permitted classification of leptons.

        :param typ: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def impact_param(sip3d, dxy, dz):
        '''
        Checks if the impact parameters of the collision are valid and sorts out
        events that do not have a clear and equal collision point.

        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def zz(z1, z2):
        '''
        Checks if the Z1 candidate and the Z2 candidate is within the allowed range.

        :param z1: float
        :param z2: float
        :return: bool
        '''
        # code

Combination of the implemented code with the partially provided classes:

In [ ]:
from include.processing.CalcAndAllowerInit import AllowedInit
from include.processing.CalcAndAllowerInit import CalcInit

AllowedInit.a_allowed_instance = AllowedStudent
AllowedInit.a_calc_instancea = CalcStudent
class Allowed(AllowedStudent, AllowedInit):
    pass
    
CalcInit.c_allowed_instance = Allowed
CalcInit.c_calc_instance = CalcStudent
class Calc(CalcStudent, CalcInit):
    pass

# Application of filters and reconstruction on MC - simulations
---------------------------------------

The existing implementations of the cuts should be tested using the background MC simulations. This is to prevent the targeted work towards a certain goal (actual measurement), since a small number of later events in the actual measurement should avoid the fact of subjectively selecting specific events in an area as far as possible.

<div class="alert alert-info">

Now apply all the cuts you have implemented to the MC simulations and measurements.
    
</div>

Therefore it is useful to create a list of tuples:

In [ ]:
from include.processing.ApplyHelper import ProcessHelper


# All Background MC and signal MC for m_H = 125 GeV
mc_files = not True
# Signal MC for other masses m = [115, 120, 122, 124, 128, 130, 135, 140, 145, 150]
other_mc_files = not True
# actual measurement
run_files = True

# folder with initial records: measurement
dir_measurement = "data/ru_aftT/"
# folder with initial records: backround MC and singal (125 GeV) MC
dir_mc = "data/mc_aftT/"
# folder with initial records: other masses
dir_other_mc = "data/other_mc/"

file_tuples = []
if mc_files:
    file_tuples += ProcessHelper.create_tuple(dir_mc)    
if other_mc_files:
    import os
    list_of_other_mc = []
    for name in os.listdir(dir_other_mc):
        file_tuples += ProcessHelper.create_tuple(os.path.join(dir_other_mc, name, "mc_aftT/"))
if run_files:
    file_tuples += ProcessHelper.create_tuple(dir_measurement)

A single such `namedtuple` contains the file and particle type of the data set (necessary for `Apply`):

In [ ]:
file_tuples[0]

The whole filter and reconstruction process can be combined in one function. The sequence of the filters is critical for the runtime.

The following filter and reconstruction steps are available for selection:

In [ ]:
Apply.help()

The function can have the following form:

In [ ]:
def filter_and_reco_process(used_pair):
    process = Apply(input_=used_pair.name, 
                    particle_type=used_pair.particle, 
                    multi_cpu=True, use_swifter=False,
                    calc_instance=Calc, 
                    allowed_instance=Allowed)
    
    # Logische Reihenfolge Wählen
    # quicksave: Speichert den Datensatz NACH dem Anwenden 
    # des Filter- bzw Rekonstruktionsschrittes
    
    process.filter(filter_name="check_type",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftT"))
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_min_pt")
    process.filter(filter_name="check_impact_param")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_exact_pt")
    process.filter(filter_name="check_m_2l")
    process.filter(filter_name="check_rel_iso")
    process.filter(filter_name="check_q")
    if process.particle_type != "muon":
        process.filter(filter_name="check_misshit")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_eta")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_m_4l",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "befZ"))
    process.reconstruct(reco_name="zz", 
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftZ"))
    process.reconstruct(reco_name="mass_4l_out_zz",
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftH"))
    del process

<div class="alert alert-info">

Select the appropriate sequence of filter and reconstruction steps.
    
(To estimate the speed with which the individual filters are executed on a data set, it is recommended to use the data set of the background MC simulations and use the following 'kwargs' when initializing the `Apply` instance: `multi_cpu=False, use_swifter=True`)
    
</div>

The application of `filter_and_reco_process` to the records: 

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output

for pair in tqdm(file_tuples):
    filter_and_reco_process(pair)
    clear_output()

# Parameterization of the background
------------------

To avoid fluctuations, it is advantageous to express the background distribution of the MC simulations by a probability density function (pdf).

<div class="alert alert-info">

Use a suitable model to parameterize the background. Also make sure that the model you choose is the most suitable one.
    
</div>

The implementation of linear combinations of the legend trepolynomials has already been carried out and summarized in the class `LLC`. Independent functions can also be used.

In [ ]:
import matplotlib.pyplot as plt
from include.parameterization.LegendrePoly import LegendreLinearCombination as LLC

In [ ]:
llc_background = LLC(x_mean=0.0) # centering around x_mean

x_ = np.linspace(-1, 1, 1000)
plt.plot(x_, llc_background.grade_0(x_, a=1))
plt.plot(x_, llc_background.grade_1(x_, a=1, b=1))
plt.plot(x_, llc_background.grade_2(x_, a=1, b=1, c=1))

In preparation, the number of bin numbers and the considered interval should be selected:

In [ ]:
bins, hist_range, info = 9999, (9999, 9999), [["2012"], ["A-D"]]
mc_dir = "./data/mc_aftH/"

Variant:

 1. <ins>The data from the histogram is raw output.</ins> 
<div class="alert alert-info">
     
    * Calculate the uncertainties on the individual entries of the unscaled histogram
    * Scale the histograms of the individual channels and combine them (and the calculated combined uncertainties) in a common histogram.

    (The individual parts can be accessed via the 'data' attribute, where it is a nested 'dict'. The first level contains the three individual channels. Each of the channels then contains the unscaled histogram entries, the correction factor and the data used for histogram creation).

</div>

In [ ]:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="background")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_ZZ_4el_2012"]

In [ ]:
# parametrization
import kafe2 as K2
# code

2. <ins>The data from the histogram is a summarized output</ins> 
    <div class="alert alert-info">
     
    Use the already combined values for a histogram fit.


</div>

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
# parametrization

The actual fit is done by using the histogram fit implemented in 'kafe2'. An example of a histogram fit can be found [here](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

It is advantageous to create a function or class to avoid lengthy repetitions.

# Selection of the appropriate signal MC
-----------------------------------------

The goal of this section is to narrow down the possible signal MC simulations and to decide on a single one and continue working with it.

## Preliminary parameterization of signal MC
------------------

Different signal MC simulations are available. These were roughly parameterized by a Gaussian distribution $\mathcal{G} \left( m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}} \right)$. The aim is to determine the dependence of the width on the invariant mass and to describe it by a suitable model. The resolution of the detector's transversal pulse is the biggest influence here.


<div class="alert alert-info">

Select the most suitable Signal-MC simulation from a given set of Signal-MC simulations and justify your choice.
</div>

The results of these Gaussian parameterizations are summarized in a Pandas Dataframe.

In [ ]:
import pandas as pd
df_ = pd.read_csv("./data/for_p4/.csv")
df_
sigma, sigma_err, mu, mu_err = tuple(df_[["sigma", "sigma_err", "mu", "mu_err"]].values.T)

<div class="alert alert-info">

Parameterize $\sigma_{\mathrm{G}}(m_{4\ell})$ using the existing data.
  
</div>

In [ ]:
def sigma_model(x, *args):
    """
    Model to for sigma parameterization
    
    """
    # code
    
    return x

# code

## Likelihood-ratio test
-----------------

The likelihood ratio test is explicitly used for the selection between the different simulations.
For this purpose, the null hypothesis $H_0$, which only corresponds to the background, and the counter hypothesis $H_1$, which corresponds to the background and a signal, are compared with each other.

The (extended) (unbinned) likelihood of the $H_0$ hypothesis is based on the parameterization of the background $f_{\mathrm{U}}(x|\theta_{\mathrm{U}})$ where $f_{\mathrm{U}}$ is the previously determined model from the background parameterization with the optimal parameters $\theta_{\mathrm{U}}$.

The distribution used for the alternative hypothesis can be described by $$f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) = \mu f_{\mathrm{S}}(m_{4\ell}|\theta_{\mathrm{S}}) + f_{\mathrm{U}}(m_{4\ell}|\theta_{\mathrm{U}}) \, , \\ \int_{\Omega} f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) \mathrm{d}m_{4\ell} = 1 $$ $f_{\mathrm{S}}(x|\theta_{\mathrm{S}})$ is a Gaussian distribution $\mathcal{G}(m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}}(m_{\mathrm{H}} ))$ with fixed values for the mass and width. For the estimator $\mu$ the following holds: $$\mu \geq 0 \, .$$ This restriction leads to a negligible distortion and is introduced for physical considerations, since in the distribution of invariant masses a excess of events is expected if a Higgs boson is present.

Thus, the quantity to be considered - the ratio of the likelihoods of both hypotheses - can be formed. For numerical stability, the negative logarithm of this is formed. The factor two is necessary for a later used asymptotic form.

$$ q_0 = -2 \ln \left( \frac{\mathcal{L}(x|\mu = \hat{\mu})}{\mathcal{L}(x|\mu = 0)} \right) \, .$$ 
$\hat{\mu}$ here is the ideal estimator for a fixed mass and width of the signal hypothesis.

In [ ]:
lower_bound, upper_bound = hist_range
mass_array = np.linspace(lower_bound, upper_bound, 100)
sigma_model_arguments = tuple([])
sigma_array = sigma_model(mass_array, *sigma_model_arguments)

<div class="alert alert-info">

Implement the likelihood ratio test. For numerical integration, you can use the ['scipy.integrate.quad'](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) method. For minimizing the likelihood function you can use ['iminuit'](https://nbviewer.jupyter.org/github/scikit-hep/iminuit/blob/master/tutorial/basic_tutorial.ipynb)
).

</div>

In [ ]:
# measurement
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 9999, (106, 151), [["2012"], ["A-D"]]
mc_dir = "../data/mc_aftH/" 
ru_dir = "../data/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
data = raw_data.data
data

# scale factors for the mc simulations
b_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
_, _, background_hist, _ = b_mc_d.variables_for_hist_fit
scale_b_mc = np.sum(background_hist)

s_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="signal", mc_dir=mc_dir)
_, _, signal_hist, _ = s_mc_d.variables_for_hist_fit
scale_s_mc = np.sum(signal_hist)

scale_s_mc, scale_b_mc

In [ ]:
# Iminuit Example:
from iminuit import Minuit

def extended_likelihood(mu=0.0):
    # FUnction that need to be minimized
    #code
    return mu

m = Minuit(extended_likelihood, 
           mu=0.5, error_mu=0.1, limit_mu=(0.0, 3.0),
           errordef=0.5)
m.migrad()
best_mu = m.values["mu"]


q0 = np.zeros(len(mass_array))

#code

The asymptotic form of the probability density of $q_0$ when no signal is expected is described by
$$ f(q_0|\hat{\mu}=0) \rightarrow \frac{1}{2}\left( \delta (q_0) + \chi^2_{1} \right) \, .$$
from this the $p_0$ value can be calculated according to:
$$ p_0 = \int\limits_{q_{0_{\mathrm{obs}}}}^{\infty} \mathrm{d}q_0 f(q_0|\hat{\mu}=0) \, . $$
using this, the significance can be estimated as follows: $$Z = \Phi^{-1}(1-p_0) = \sqrt{q_{0_{\mathrm{obs}}}} \, .$$ However, in this case it should only be considered as a first estimation because the parameterization of the input signal is not accurate.
Likewise, the systematics are not considered at this point, only the statistical uncertainty is included in the calculation of $p_0$ or $Z$.

Saving the sizes determined in this way can be done in the form of a panda dataframe.

In [ ]:
q0, p0 = np.zeros(len(sigma_array)), np.zeros(len(sigma_array))
df = pd.DataFrame(data=np.array([mass_array, sigma_array, q0, p0, np.sqrt(q0)]).T,
                  columns=["mass_array", "gauss_sigma", "q0", "p0", "sqrt_q0"])

<div class="alert alert-info">

Visualize the results. For the significance levels the 'get_p0_sigma_lines' from 'PlotHelper' can be used.
</div>

In [ ]:
import matplotlib.pyplot as plt
from include.stattest.PlotHelper import PlotHelper

In [ ]:
fig, ax = plt.subplots(1,1)
PlotHelper.get_p0_sigma_lines(x_=mass_array, 
                             ax_obj_=ax, 
                             max_sigma_=4)
ax.plot(df.mass_array, df.p0+1)  # <- change this line accordingly
plt.show()

# Examination of the final distributions
-----------------------

After the selection of the appropriate signal distribution, different quantities can finally be displayed.

<div class="alert alert-info">

Consider at least the distribution of the four lepton invariant masses, as well as the masses of the two Z-bosons.
    
</div>

Furthermore, the following sizes can also be displayed, although it is questionable whether `z1_index`, `z2_index`, as well as `z1_tag` or `z2_tag` are sizes that require detailed visual observation.

In [ ]:
print(ProcessHelper.print_possible_variables("data/mc_aftH/MC_2012_H_to_ZZ_to_4L_2el2mu_aftH.csv"))

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="data/mc_aftH/", 
           ru_dir="data/ru_aftH/")  

In [ ]:
# example
h.variable("energy", 50, (0, 200))
ax = plt.gca()
ax.set_xlabel(r"$p_T$ in GeV")
ax.set_ylabel("Bineinträge")
plt.show()

In [ ]:
# other histograms

# Parameterization of the signal MC
--------------------------------------------------------

After choosing the best suited signal hypothesis, the distribution will be parameterized more precisely. This is done analogously to the background parameterization.

<div class="alert alert-info">

Parameterize the signal distribution with a suitable model. Also make sure that the model you choose is the most suitable one.

</div>

The students can choose from distributions that have already been implemented:
- Gauss distribution
- Cauchy distribution
- Voigt Profile
- Single - Sided - Crystal - Ball
- Double - Sided - Crystal - Ball


However, independently implemented distributions can also be used.

In [ ]:
from include.parameterization.SignalFunction import SignalFunction as SF

variant:

 1. <ins>The data from the histogram is raw output.</ins> 
<div class="alert alert-info">
     
    * Calculate the uncertainties on the individual entries of the unscaled histogram
    * Scale the histograms of the individual channels and combine them (and the calculated combined uncertainties) in a common histogram.

    (The individual parts can be accessed via the 'data' attribute, where it is a nested 'dict'. The first level contains the three individual channels. Each of the channels then contains the unscaled histogram entries, the correction factor and the data used for histogram creation).

</div>

In [ ]:
bins, hist_range, info = 9999, (9999, 9999), [["2012"], ["A-D"]]
mc_dir = "./data/mc_aftH/"

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="signal")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_H_ZZ_4el_2012"]

In [ ]:
# parametrization 
import kafe2 as K2
# code

2. <ins>The data from the histogram is a summarized output</ins> 
    <div class="alert alert-info">
     
    Use the already combined values for a histogram fit.


</div>

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

The actual fit is done by using the histogram fit implemented in 'kafe2'. An example of a histogram fit can be found [here](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

It is advantageous to create a function or class to avoid lengthy repetitions.

In [ ]:
# parametrization
import kafe2 as K2
# code

# Determination of statistical significance
-------------------------

After successful parameterization of the background and the signal, the significance of the signal can be determined. The following parameterization is selected for this purpose:
$$ f(x, \alpha_\mathrm{S}) = \alpha_\mathrm{S} f_{\mathrm{S}}(x|\theta_{\mathrm{S}}) + (1 - \alpha_\mathrm{S}) f_{\mathrm{U}}(x|\theta_{\mathrm{U}}) \, .$$

The optimal parameter $\alpha_{\mathrm{S}}$ corresponds to the signal probability of this measurement. This determination is carried out with the help of the $-2\ln\mathcal{L(\alpha_{\mathrm{S}})}$ function which is to be implemented independently. The previously determined parameters $\theta_{\mathrm{S}}$ and $\theta_{\mathrm{U}}$ of the probability density function of the signal or background are fixed for the determination (profiled Likelihood).

The evaluation of the function value $-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)$ should be emphasized here. This corresponds to the assumption that there is no signal component in the existing measurement and the measurement can therefore be explained by the null hypothesis (only background). In asymptotic form, therefore, the significance can be determined analogously to the expression in the chapter on parameterization of the signal MC expression with: $$ Z = \sqrt{\frac{-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)}{\mathcal{L}_{\mathrm{min}}}} \, .$$

<div class="alert alert-info">
     
Implement the $-2\ln\mathcal{L(\alpha_S, m_{4\ell})}$ function (if not already present).Determine the signal probability plus uncertainty of the measurement.  Specify the significance with which a higgs-like boson is measured. In addition, vary the mass of the higgs-like boson for the two-dimensional variant and compare it with the literature. Visualize and discuss your results.
    
For the minimization you can use `minuit` as in QRT. The numerical integration can again be done using `scipy.integrate.quad`.
    
It is also possible to perform an `UninnedFit` using `kafe2` and implement the extended likelihood as a cost function independently.
</div>

In [ ]:
# measurement:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 9999, (106, 151), [["2012"], ["A-D"]]
mc_dir = "./data/mc_aftH/" 
ru_dir = "./data/mc_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
raw_data.data

In [ ]:
# code iminuit

def extended_nll_iminuit(alpha_s, mass=125.0):
    # code
    return mass

In [ ]:
# code kafe2

def extended_nll_kafe2(data, model, parameter_values, parameter_constraints):
    """
    
    :param data: measurement array
    :param model: model array (calculate individual in # code)
    :param parameter_values:
    :param parameter_constraints: (not used)
    :return:
    """
    
    # check if one of the parameter_values is nan
    if any(f"{par}" == "nan" for par in parameter_values):
        return np.inf
    
    # your code
    
    _total_log_likelihood = 0.0 # your code
    
    # check if _total_log_likelihood is nan
    if np.isnan(_total_log_likelihood):
        return np.inf
    return -2.0 * _total_log_likelihood

## 1D profiled Likelihood
-----------------------------------

The one-dimensional variant only varies the signal probability. The mass is held after the parameterization and is not varied anymore.

### Iminuit variant

In [ ]:
# code (iminuit minimization)

visualization:

In [ ]:
from include.stattest.PlotHelper import PlotHelper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
alpha_ = np.linspace(0.0, 0.8, 1000)
nll_ = extended_nll(alpha_s=alpha_, mass=125)

# as example (to be replaced with actual nll array)
nll_ = 150*(alpha_ - 0.4) ** 2

z_ = np.sqrt(nll_[0])

# code to further customize the plot

fig, ax = plt.subplots(1,1)
PlotHelper.plot_nll_scan_main_window(ax_obj_=ax, 
                                     x_=alpha_, 
                                     y_=nll_, 
                                     z_=z_)

ax_in = inset_axes(ax, width='50%', height='40%', loc="upper right")
PlotHelper.plot_nll_scan_inside_window(ax_obj_=ax_in, 
                                       x_=alpha_, y_=nll_, 
                                       x_ticks_=PlotHelper.get_error_points_from_nll(x_=alpha_, nll_=nll_))
mark_inset(ax, ax_in, loc1=2, loc2=4, fc="none", ec="grey", lw=0.5, alpha=0.5)
plt.show()

### kafe2 variant

For the 'kafe2' variant, an 'uninned container' must be created and adjusted. For the 1D variant it is recommended to use the method `fix_parameter(par_name, value)`.
The significance can be calculated using the implemented `extended_nll_kafe2` cost function. It is recommended that you use the `limit_parameter` to restrict the parameter `alpha` to a reasonable interval.

In [ ]:
import kafe2 as K2

my_cost_function = K2.UnbinnedCostFunction_UserDefined(extended_nll_kafe2)

# more code

## 2D profiled Likelihood - Mass determination
-------------------

Finally, the mass is also determined. This is done by the two-dimensional variation - mass and signal probability.

### Iminuit variant

Similar to the 1D version, the best mass can be determined at the same time. The procedure is analogous to the 1D variant/QRT implementation.

In [ ]:
# code:
#     - minimization
#     - plots

### kafe2 variant

The only difference to the 1D variant is that the mass is no longer fixed by `fix_parameter`, but is varied in a certain interval (`limit_parameter`).

In [2]:
# code

## Combination of the measurements: CMS and ATLAS
----------------------------

In order to obtain a better statistical significance, it is advantageous to include further measurements in the overall assessment. This can be done either by adding more data or by another experiment. Within the scope of this task the second option is performed.

For the ATLAS experiment, therefore, similar steps as above would have to be carried out, although many of the cuts show deviations. To shorten the work a bit, the final measurement is provided, as well as the histogram entries of the MC simulations.

In addition, the parameters of the most suitable functions (linear combination of centered legend trepolynomials up to the third order for the background and the DSCB distribution for the signal) are provided (but it is also possible to check these results yourself).

In [ ]:
import numpy as np
import pandas as pd
import kafe2 as K2
import os
from pprint import pprint

In [ ]:
atlas_data_dir_ = "../data/for_p4/"
atlas_files = [os.path.join(atlas_data_dir_, file) for file in os.listdir(atlas_data_dir_) if ".csv" in file and "atlas" in file]
pprint(atlas_files)
hist_range_ = (106, 151)
num_bins = 45

data_A = np.loadtxt(atlas_files[2])
data_A = data[(data >= hist_range_[0]) & (data <= hist_range_[1])]

mc_bac = np.loadtxt(atlas_files[1], delimiter=",").T
mc_bac_data = mc_bac[0]
mc_bac_error = mc_bac[1]

mc_sig = np.loadtxt(atlas_files[0], delimiter=",").T
mc_sig_data = mc_sig[0]
mc_sig_error = mc_sig[1]

In [ ]:
data_CMS = data
data_ATLAS = data_A

background_parameter_CMS = background_parameter
signal_parameter_CMS = signal_parameter

background_parameter_ATLAS  = np.array([0.025050083639971264, 
                                        0.00018744120591615378, 
                                        -1.178181555551107e-05, 
                                        1.1872474265844408e-07])
signal_parameter_ATLAS = np.array([2.1395802886491615, 
                                   124.19274632605674, 
                                   0.8324353801486924, 
                                   1.8034218165656613, 
                                   14.109501070824912, 
                                   19.9999999999894])
bfu_ATLAS = llc_background.grade_3
sfu_ATLAS = sf.DSCB

<div class="alert alert-info">
Combine the two measurements and determine the mass of the Higgs boson resulting from the combination, the signal probability, as well as the significance that a Higgs boson was measured in the recordings.

You can reuse/modify the functions you have already implemented.
    
Note:
    $$ \mathcal{L_{\mathrm{tot}}(\alpha_S, m_{4\ell})} = \mathcal{L_{1}(\alpha_S, m_{4\ell})} \mathcal{L_{2}(\alpha_S, m_{4\ell})} $$
</div>

In [ ]:
# code (1D and 2D minimization)